# Setup

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import time
import nltk
from nltk.corpus import stopwords
# from nltk.stem import French
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop
!pip install git+https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer # package for french lemmatization
from nltk.tokenize.treebank import TreebankWordDetokenizer
import random
import re
!pip install langdetect
from nltk.corpus import wordnet
from langdetect import detect
!pip install tqdm
from tqdm import tqdm
from itertools import product


seed = 10
random.seed(seed)
np.random.seed(seed)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


  Cloning https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git to /tmp/pip-req-build-dw10e7b2
  Running command git clone --filter=blob:none --quiet https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git /tmp/pip-req-build-dw10e7b2
  Resolved https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git to commit bc0ebd0135a6cc78f48ddf184069b4c0b9c017d8
  Preparing metadata (setup.py) ... done
  Created wheel for FrenchLefffLemmatizer: filename=FrenchLefffLemmatizer-0.3-py3-none-any.whl size=3533513 sha256=5780f364e2b53f908b42c56e443b4c9a88b783ff2fb8f2433cfd536847d28857
  Stored in directory: /tmp/pip-ephem-wheel-cache-12h5jdht/wheels/99/54/81/553bffdc8e0781c925f9a26606894935adb29b56175a9a358b
Successfully built FrenchLefffLemmatizer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=015c31d1bd6e8cc731d987cf99f78

## Get Data

In [3]:
path = '/content/drive/MyDrive/Olympus in the Sky/McGill/2024 - Winter/ECSE 551/Data/'
df = pd.read_csv(path + 'train.csv', encoding = "ISO-8859-1")
df = df.sample(frac=1, random_state=1).reset_index(drop = True)
df = df[~df['body'].str.contains('\?'*10)]

training_data = df['body'].apply(lambda x: x.lower().replace('_', ' ')).to_numpy()
training_labels = df['subreddit'].to_numpy()

lemmatized_training_data = pd.read_csv(path + 'lemmatized.csv')['Body'].to_numpy()

In [4]:
path = '/content/drive/MyDrive/Olympus in the Sky/McGill/2024 - Winter/ECSE 551/Data/'
df_swapped = pd.read_csv(path + 'train.csv', encoding = "ISO-8859-1")
df_swapped = df_swapped.sample(frac=1, random_state=1).reset_index(drop = True)
df_swapped = df_swapped[~df_swapped['body'].str.contains('\?'*10)]

training_data_swapped   = df_swapped['body'].apply(lambda x: x.lower().replace('_', ' ').replace('\x82', 'e').replace('\x85', 'a').replace('\x87', 'c').replace('\x8a', 'e').replace('\x88', 'e').replace('\x93', 'a').replace('\x80', 'l').replace('\x96', 'o')).to_numpy()
training_labels = df_swapped['subreddit'].to_numpy()

lemmatized_training_data_swapped = pd.read_csv(path + 'lemmatized_swapped.csv')['body'].to_numpy()

## Preprocessing

In [5]:
def get_wordnet_pos(word):
  """Map POS tag to first character lemmatize() accepts"""
  tag = nltk.pos_tag([word])[0][1][0].upper()
  tag_dict = {"J": wordnet.ADJ,
              "N": wordnet.NOUN,
              "V": wordnet.VERB,
              "R": wordnet.ADV}
  return tag_dict.get(tag, wordnet.NOUN)

def remove_stop_words(arr_of_words, stop_words):
  return [word for word in arr_of_words if word not in stop_words]

def lemmatize(arr_of_words):
  lang = detect(' '.join(arr_of_words))
  if lang == 'fr':
      lemmatizer = FrenchLefffLemmatizer()
  else:
      lemmatizer = WordNetLemmatizer()

  return [lemmatizer.lemmatize(word, pos = get_wordnet_pos(word)) for word in arr_of_words]

def preprocess(sentences, regex_pattern = r"(?u)\b([a-z][a-z]+)\b", stop_words = None, is_lemmatize = False):
  new_sentence_list = []
  pattern = regex_pattern

  for iter, sentence in enumerate(sentences):

      words = list(re.findall(pattern, sentence))
      if stop_words:
        words = remove_stop_words(words, stop_words)

      if is_lemmatize:
        words = lemmatize(words)

      new_sentence_list.append(' '.join(words))

  return np.array(new_sentence_list)

# NB and K-Fold

In [ ]:
# to find P(Y=k), just do test_labels.where(k)/len(test_labels)
# vectos_train will contain all the information I need to create the counting for each word
# use the get_feature_names to create a dictionary to map to all counts
# this dictionary would be contained in another dict that has the labels as keys
# to find P(xj,k), find the indices where test_labels = (k), then find

In [ ]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

class LemmaTokenizer:
     def __init__(self, pattern):
       self.pattern = pattern
     def __call__(self, doc):
       doc = doc.lower().replace('_', ' ')
       doc_words = list(set(re.findall(self.pattern, doc)))
       lang = detect(' '.join(doc_words))
       if lang == 'fr':
           lemmatizer = FrenchLefffLemmatizer()
       else:
           lemmatizer = WordNetLemmatizer()
       return [lemmatizer.lemmatize(word,pos =get_wordnet_pos(word)) for word in doc_words]

In [6]:
class NB():
  def __init__(self, max_features = 3000, token_pattern = r"(?u)\b([a-z][a-z]+)\b", min_df = 1, max_df = 1.0, ngram_range = (1,1), alpha = 1):
    spacy_stopwords_list = list(fr_stop) + list(en_stop)
    nltk_stopwords_list = stopwords.words('english') + stopwords.words('french')
    self.pattern = token_pattern
    self.ngram_range = ngram_range
    self.alpha = alpha
    self.vectorizer = CountVectorizer(binary = True, max_features = max_features, token_pattern = self.pattern, min_df = min_df, max_df = max_df, ngram_range = ngram_range)
    self.no_pred = []

  # currently train_corpus expects a np array containing a list of strings, not array of words, nxm
  # test_label is an nx1 array
  def fit(self, train_corpus, test_labels):
    assert len(train_corpus)  == len(test_labels)
    assert type(train_corpus) == type(test_labels) == np.ndarray

    vectors_train       = self.vectorizer.fit_transform(train_corpus).todense()
    self.analyzer       = self.vectorizer.build_analyzer()
    self.word_list      = self.vectorizer.get_feature_names_out()
    self.num_samples    = len(test_labels)
    self.word_count     = dict(zip(self.word_list, np.array(vectors_train.sum(axis=0))[0]))

    unique, counts   = np.unique(test_labels, return_counts=True)
    self.label_count = dict(zip(unique, counts))
    self.labels      = unique

    self.word_count_given_label = {}
    for label in self.labels:
      indices = np.where(test_labels == label)[0]
      tot_word_count = np.array(vectors_train[indices].sum(axis=0))[0]
      self.word_count_given_label[label] = {self.word_list[i] : tot_word_count[i] for i in range(len(tot_word_count))}
      # print(label, self.word_count_given_label[label])

  # Assuming test_corpus is 2-d array where each test sample is a string
  def predict(self, test_corpus):

    predictions = []
    for corpus in tqdm(test_corpus, desc="Loading…", ascii=False, ncols=75):

      best_label = ''
      best_prob  = -np.inf

      corpus_words = self.analyzer(corpus)

      for label in self.labels:
        p_of_y = self.label_count[label]/self.num_samples

        p_of_x_given_y = 1

        for word in self.word_list:
          if word in corpus_words:
            xj = 1
          else:
            xj = 0

          theta_xj_k      = (self.word_count_given_label[label][word] + self.alpha) / (self.label_count[label] + self.alpha*len(self.labels))
          p_of_x_given_y *= (theta_xj_k**(xj) * (1-theta_xj_k)**(1-xj))
          # print(label, word, p_of_x_given_y, theta_xj_k, (theta_xj_k**(xj) * (1-theta_xj_k)**(1-xj)))

        p_of_y_given_x = np.log(p_of_y * p_of_x_given_y)
        # print(p_of_y, p_of_x_given_y)
        # print(label, p_of_y_given_x)

        # print(p_of_y_given_x, best_prob)
        if p_of_y_given_x > best_prob:
          best_prob = p_of_y_given_x
          best_label = label

      if p_of_x_given_y == 0:
        self.no_pred.append(corpus)
      #   best_label = random.choice(self.labels)

      predictions.append(best_label)

    return predictions

In [7]:
def k_fold_validation(x_train_data, y_train_data, model = NB(), K = 10):
  assert type(x_train_data) == np.ndarray and type(y_train_data) == np.ndarray

  len_of_data  = len(x_train_data) - len(x_train_data) % K
  data_x       = x_train_data[:len_of_data]
  data_y       = y_train_data[:len_of_data]
  size_of_fold = len(x_train_data[:len_of_data]) // K
  validation_error = 0

  error    = []
  pred     = []
  timings  = []
  no_preds = []

  for i in range(K):

    naive_bayes = model

    if i != K-1:
      validation_fold_x = data_x[i*size_of_fold:(i+1)*size_of_fold]
      validation_fold_y = data_y[i*size_of_fold:(i+1)*size_of_fold]

      training_folds_x  = np.concatenate((data_x[:i*size_of_fold], data_x[(i+1)*size_of_fold:]))
      training_folds_y  = np.concatenate((data_y[:i*size_of_fold], data_y[(i+1)*size_of_fold:]))

    else:
      validation_fold_x = data_x[i*size_of_fold:]
      validation_fold_y = data_y[i*size_of_fold:]

      training_folds_x  = data_x[:i*size_of_fold]
      training_folds_y  = data_y[:i*size_of_fold]

    start_time = time.time()
    naive_bayes.fit(training_folds_x, training_folds_y)
    end_time = time.time()

    timings.append(end_time - start_time)

    pred_valid = naive_bayes.predict(validation_fold_x)
    no_pred = naive_bayes.no_pred
    # pred_train = naive_bayes.predict(training_folds_x)

    fold_error = {}
    fold_error['validation'] = 1 - accuracy_score(pred_valid, validation_fold_y)
    validation_error        += 1 - accuracy_score(pred_valid, validation_fold_y)
    # fold_error['train']      = 1 - accuracy_score(pred_train, training_folds_y)

    model_pred = {}
    model_pred['validation'] = (pred_valid, validation_fold_y)
    # model_pred['train']      = (pred_train, training_folds_y )

    error.append(fold_error)
    pred.append(model_pred)
    no_preds.append(no_pred)

  info = {'error': error, 'pred': pred, 'time': timings, 'no_preds' : no_preds}

  return validation_error/K, info

# Simple test cases for NB

In [11]:
train_corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
     'Is this the first document?',]
test_labels = np.array(['1', '2', '3', '4'])
train_corpus = np.array(train_corpus)
test_corpus = ['harro first pink haha']
model = NB()
model.fit(train_corpus, test_labels)
model.predict(test_corpus)

Loading…: 100%|████████████████████████████| 1/1 [00:00<00:00, 1451.82it/s]


['1']

In [12]:
# Training data
X_train = np.array([
    "I love this movie",
    "This movie is great",
    "A movie like this is great",
    "I hate this movie",
    "This movie is terrible"])

# Corresponding labels
y_train = np.array([1, 1, 1, 0, 0])  # 1 for positive sentiment, 0 for negative sentiment

model = NB()
model.fit(X_train, y_train)
X_test = [
    "I love this movie terrible", # Example of both class being viable but the second being chosen due to label eval order in NB
    "I hate this great movie",
    "This movie is terrible"]

res = model.predict(X_test)
print()
print(res)

Loading…: 100%|████████████████████████████| 3/3 [00:00<00:00, 3534.53it/s]


[0, 1, 0]


# K-fold for NB

In [ ]:
# TODO: No predictions given, divide by zero warnings might be the cause (Solved?)
# TODO: Test changing the prob caluclation to be a sum of logs

In [32]:
# No preprocessing
test_split = 0.2

(training_data_split, testing_data_split, training_labels_split, testing_labels_split) = train_test_split(training_data_swapped, training_labels,  test_size = test_split, random_state=seed)

model = NB()
model.fit(training_data_split, training_labels_split)
pred = model.predict(testing_data_split)
print((pred == testing_labels_split).sum()/len(testing_labels_split))
print()
print('\nModel Misses: {}'.format(model.no_pred))

mean_error, info = k_fold_validation(training_data_swapped, training_labels)
print()
print('Mean Error: {}'.format(mean_error))
print()
print('\nModel Misses: {}'.format(model.no_pred))

Loading…: 100%|██████████████████████████| 280/280 [00:12<00:00, 22.41it/s]


0.5464285714285714


Model Misses: []


Loading…:  73%|██████████████████▉       | 101/139 [00:08<00:02, 18.12it/s]<ipython-input-29-8f7cd07375b2>:60: RuntimeWarning: divide by zero encountered in log
  p_of_y_given_x = np.log(p_of_y * p_of_x_given_y)
Loading…:  84%|█████████████████████▉    | 117/139 [00:03<00:00, 33.05it/s]<ipython-input-29-8f7cd07375b2>:60: RuntimeWarning: divide by zero encountered in log
  p_of_y_given_x = np.log(p_of_y * p_of_x_given_y)
Loading…:  61%|████████████████▌          | 85/139 [00:02<00:01, 29.94it/s]<ipython-input-29-8f7cd07375b2>:60: RuntimeWarning: divide by zero encountered in log
  p_of_y_given_x = np.log(p_of_y * p_of_x_given_y)
Loading…:   4%|█                           | 5/139 [00:00<00:03, 42.16it/s]<ipython-input-29-8f7cd07375b2>:60: RuntimeWarning: divide by zero encountered in log
  p_of_y_given_x = np.log(p_of_y * p_of_x_given_y)
Loading…:  68%|██████████████████▎        | 94/139 [00:03<00:01, 29.52it/s]<ipython-input-29-8f7cd07375b2>:60: RuntimeWarning: divide by zero encountere


Mean Error: 0.4467625899280575


Model Misses: []


In [37]:
# With Processing
spacy_stopwords_list = list(fr_stop) + list(en_stop)
nltk_stopwords_list = stopwords.words('english') + stopwords.words('french')
other_stop_words = ['http', 'https', 'www']
stop_words = list(set().union(spacy_stopwords_list, nltk_stopwords_list, other_stop_words))

processed_training_data = preprocess(training_data_swapped, stop_words = stop_words)

test_split = 0.2

(training_data_split, testing_data_split, training_labels_split, testing_labels_split) = train_test_split(processed_training_data, training_labels, test_size = test_split, random_state=seed)

model = NB()
model.fit(training_data_split, training_labels_split)
pred = model.predict(testing_data_split)
print((pred == testing_labels_split).sum()/len(testing_labels_split))

mean_error, info = k_fold_validation(processed_training_data, training_labels)
print()
print('Mean Error: {}'.format(mean_error))
print()
print('\nModel Misses: {}'.format(model.no_pred))

Loading…: 100%|██████████████████████████| 280/280 [00:06<00:00, 44.61it/s]


0.6357142857142857


Loading…: 100%|██████████████████████████| 139/139 [00:03<00:00, 44.31it/s]


Mean Error: 0.3654676258992805


Model Misses: []


In [8]:
spacy_stopwords_list = list(fr_stop) + list(en_stop)
nltk_stopwords_list = stopwords.words('english') + stopwords.words('french')
other_stop_words = ['http', 'https', 'www']
stop_words = list(set().union(spacy_stopwords_list, nltk_stopwords_list, other_stop_words))
print(len(list(set().union(spacy_stopwords_list, nltk_stopwords_list, other_stop_words))))
print(len(list(set().union(spacy_stopwords_list))))
print(len(list(set().union(nltk_stopwords_list))))

949
825
327


In [8]:
lemmatized_training_data_swapped = preprocess(training_data_swapped, is_lemmatize = True)

In [39]:
# With Processing
# lemmatized_data = preprocess(training_data, regex_pattern = r"(?u)\b[a-z][a-z]+\b", stop_words = None, is_lemmatize = True)
spacy_stopwords_list = list(fr_stop) + list(en_stop)
nltk_stopwords_list = stopwords.words('english') + stopwords.words('french')
other_stop_words = ['http', 'https', 'www']
stop_words = list(set().union(spacy_stopwords_list, nltk_stopwords_list, other_stop_words))

processed_training_data = preprocess(lemmatized_training_data_swapped, stop_words = stop_words)

test_split = 0.2

(training_data_split, testing_data_split, training_labels_split, testing_labels_split) = train_test_split(processed_training_data, training_labels,  test_size = test_split, random_state=seed)

model = NB()
model.fit(training_data_split, training_labels_split)
pred = model.predict(testing_data_split)
print((pred == testing_labels_split).sum()/len(testing_labels_split))

mean_error, info = k_fold_validation(processed_training_data, training_labels)
print()
print('Mean Error: {}'.format(mean_error))
print()
print('\nModel Misses: {}'.format(model.no_pred))

Loading…: 100%|██████████████████████████| 280/280 [00:06<00:00, 41.08it/s]


0.6178571428571429


Loading…: 100%|██████████████████████████| 139/139 [00:03<00:00, 44.80it/s]


Mean Error: 0.3661870503597123


Model Misses: []


In [ ]:
from sklearn.naive_bayes import MultinomialNB

spacy_stopwords_list = list(fr_stop) + list(en_stop)
nltk_stopwords_list = stopwords.words('english') + stopwords.words('french')
other_stop_words = ['http', 'https', 'www']
stop_words = list(set().union(spacy_stopwords_list, nltk_stopwords_list, other_stop_words))

# processed_training_data = preprocess(lemmatized_training_data, regex_pattern = r"(?u)\b[a-z][a-z]+\b", stop_words = stop_words)
processed_training_data = preprocess(training_data, regex_pattern = r"(?u)\b[a-z][a-z]+\b", stop_words = stop_words)

test_split = 0.1

(training_data_split, testing_data_split, training_labels_split, testing_labels_split) = train_test_split(processed_training_data, training_labels,  test_size = int(len(processed_training_data)*test_split), random_state=seed)

for mf in [500, 1000, 2000, 3000, 4000, 5000]:
  for ng in [(1,1), (1,2), (1,3), (1,5)]:
    vectorizer = CountVectorizer(binary=True, max_features = mf, token_pattern = r"(?u)\b[a-z][a-z]+\b", ngram_range=ng)
    x_train = np.array(vectorizer.fit_transform(training_data_split).todense())
    x_test  = np.array(vectorizer.transform(testing_data_split).todense())


    clf = MultinomialNB()
    clf.fit(x_train, training_labels_split)
    print(mf, ng)
    print(clf.score(x_test, testing_labels_split))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

spacy_stopwords_list = list(fr_stop) + list(en_stop)
nltk_stopwords_list = stopwords.words('english') + stopwords.words('french')
other_stop_words = ['http', 'https', 'www']
stop_words = list(set().union(spacy_stopwords_list, nltk_stopwords_list, other_stop_words))

processed_training_data = preprocess(lemmatized_training_data, stop_words = stop_words)
# processed_training_data = preprocess(training_data, stop_words = stop_words)

model = LogisticRegression()
# k_fold = KFold(n_splits=10, shuffle=True, random_state=seed)  # Define number of folds (k) and random state

pipeline = Pipeline([
    ('vectorizer', None),
    ('model', model)
])

# Define the parameter grid with conditional hyperparameters for both vectorizers
param_grid = [
    {
        'vectorizer': [TfidfVectorizer(token_pattern = r"(?u)\b[a-z][a-z]+\b")],
        'vectorizer__max_features': [None, 3000, 4000],
        'vectorizer__max_df': [0.5, 1.0],  # Example TfidfVectorizer hyperparameter
        'vectorizer__min_df': [1, 2],          # Example TfidfVectorizer hyperparameter
        'vectorizer__ngram_range': [(1,1), (1,2), (1,3)],
        'model__C': [0.1, 1, 10],
    },
    {
        'vectorizer': [CountVectorizer(token_pattern = r"(?u)\b[a-z][a-z]+\b")],
        'vectorizer__max_features': [None, 3000, 4000],
        'vectorizer__ngram_range': [(1,1), (1,2), (1,3)],
        'vectorizer__binary': [True, False],
        'vectorizer__max_df': [0.5, 1.0],
        'vectorizer__min_df': [1, 2],
        'model__C': [0.1, 1, 10],
    }
]

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=10, scoring='accuracy')
grid_search.fit(processed_training_data, training_labels)

# Get the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)


In [38]:
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Best Parameters: {'model__C': 10, 'vectorizer': TfidfVectorizer(max_df=0.5, max_features=4000,
                token_pattern='(?u)\\b[a-z][a-z]+\\b'), 'vectorizer__max_df': 0.5, 'vectorizer__max_features': 4000, 'vectorizer__min_df': 1, 'vectorizer__ngram_range': (1, 1)}
Best Score: 0.6961776753712238


In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

spacy_stopwords_list = list(fr_stop) + list(en_stop)
nltk_stopwords_list = stopwords.words('english') + stopwords.words('french')
other_stop_words = ['http', 'https', 'www']
stop_words = list(set().union(spacy_stopwords_list, nltk_stopwords_list, other_stop_words))

processed_training_data_1 = preprocess(lemmatized_training_data, stop_words = stop_words)
processed_training_data_2 = preprocess(training_data, stop_words = stop_words)

pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(token_pattern = r"(?u)\b[a-z][a-z]+\b", max_df = 0.5, max_features = 4000, min_df= 1)),
    ('model', LogisticRegression(random_state = seed, max_iter = 1000))
])

param_grid = [

    {
        'model__C': [1e-3, 1e-1, 1, 10, 1e2],
        'model__solver' : ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
    }
]

for x_train in (processed_training_data_1, processed_training_data_2):
  grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='accuracy')
  grid_search.fit(x_train, training_labels)

  # Get the best parameters and best score
  print("Best Parameters:", grid_search.best_params_)
  print("Best Score:", grid_search.best_score_)


Best Parameters: {'model__C': 10, 'model__solver': 'lbfgs'}
Best Score: 0.6961776753712238
Best Parameters: {'model__C': 10, 'model__solver': 'liblinear'}
Best Score: 0.6890527393753201


In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

spacy_stopwords_list = list(fr_stop) + list(en_stop)
nltk_stopwords_list = stopwords.words('english') + stopwords.words('french')
other_stop_words = ['http', 'https', 'www']
stop_words = list(set().union(spacy_stopwords_list, nltk_stopwords_list, other_stop_words))

processed_training_data = preprocess(lemmatized_training_data_swapped, stop_words = stop_words)
# processed_training_data = preprocess(training_data, stop_words = stop_words)

model = LogisticRegression()
# k_fold = KFold(n_splits=10, shuffle=True, random_state=seed)  # Define number of folds (k) and random state

pipeline = Pipeline([
    ('vectorizer', None),
    ('model', model)
])

# Define the parameter grid with conditional hyperparameters for both vectorizers
param_grid = [
    {
        'vectorizer': [TfidfVectorizer(token_pattern = r"(?u)\b[a-z][a-z]+\b")],
        'vectorizer__max_features': [None, 3000, 4000],
        'vectorizer__max_df': [0.5, 1.0],  # Example TfidfVectorizer hyperparameter
        'vectorizer__min_df': [1, 2],          # Example TfidfVectorizer hyperparameter
        'vectorizer__ngram_range': [(1,1), (1,2), (1,3)],
    },
    {
        'vectorizer': [CountVectorizer(token_pattern = r"(?u)\b[a-z][a-z]+\b")],
        'vectorizer__max_features': [None, 3000, 4000],
        'vectorizer__ngram_range': [(1,1), (1,2), (1,3)],
        'vectorizer__binary': [True, False],
        'vectorizer__max_df': [0.5, 1.0],
        'vectorizer__min_df': [1, 2],
    }
]

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=10, scoring='accuracy')
grid_search.fit(processed_training_data, training_labels)

# Get the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)


Best Parameters: {'vectorizer': TfidfVectorizer(max_df=0.5, min_df=2, token_pattern='(?u)\\b[a-z][a-z]+\\b'), 'vectorizer__max_df': 0.5, 'vectorizer__max_features': None, 'vectorizer__min_df': 2, 'vectorizer__ngram_range': (1, 1)}
Best Score: 0.6783077316948285


In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

spacy_stopwords_list = list(fr_stop) + list(en_stop)
nltk_stopwords_list = stopwords.words('english') + stopwords.words('french')
other_stop_words = ['http', 'https', 'www']
stop_words = list(set().union(spacy_stopwords_list, nltk_stopwords_list, other_stop_words))

processed_training_data_1 = preprocess(lemmatized_training_data_swapped, stop_words = stop_words)
processed_training_data_2 = preprocess(training_data_swapped, stop_words = stop_words)

pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(max_df=0.5, min_df=2, token_pattern='(?u)\\b[a-z][a-z]+\\b')),
    ('model', LogisticRegression(random_state = seed, max_iter = 1000))
])

param_grid = [

    {
        'model__C': [1e-3, 1e-1, 1, 10, 1e2],
        'model__solver' : ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
    }
]

for x_train in (processed_training_data_1, processed_training_data_2):
  grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=10, scoring='accuracy')
  grid_search.fit(x_train, training_labels)

  # Get the best parameters and best score
  print("Best Parameters:", grid_search.best_params_)
  print("Best Score:", grid_search.best_score_)


Best Parameters: {'model__C': 10, 'model__solver': 'liblinear'}
Best Score: 0.6947491039426523
Best Parameters: {'model__C': 10, 'model__solver': 'liblinear'}
Best Score: 0.6997721454173067


In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

spacy_stopwords_list = list(fr_stop) + list(en_stop)
nltk_stopwords_list = stopwords.words('english') + stopwords.words('french')
other_stop_words = ['http', 'https', 'www']
stop_words = list(set().union(spacy_stopwords_list, nltk_stopwords_list, other_stop_words))

processed_training_data = preprocess(lemmatized_training_data_swapped, stop_words = stop_words)
# processed_training_data = preprocess(training_data, stop_words = stop_words)

model = LogisticRegression()
# k_fold = KFold(n_splits=10, shuffle=True, random_state=seed)  # Define number of folds (k) and random state

pipeline = Pipeline([
    ('vectorizer', None),
    ('model', model)
])

# Define the parameter grid with conditional hyperparameters for both vectorizers
param_grid = [
    {
        'vectorizer': [TfidfVectorizer(token_pattern = r"(?u)\b[a-z][a-z]+\b")],
        'vectorizer__max_features': [None, 3000, 4000],
        'vectorizer__max_df': [0.5, 1.0],  # Example TfidfVectorizer hyperparameter
        'vectorizer__min_df': [1, 2],          # Example TfidfVectorizer hyperparameter
        'vectorizer__ngram_range': [(1,1), (1,2), (1,3)],
    },
    {
        'vectorizer': [CountVectorizer(token_pattern = r"(?u)\b[a-z][a-z]+\b")],
        'vectorizer__max_features': [None, 3000, 4000],
        'vectorizer__ngram_range': [(1,1), (1,2), (1,3)],
        'vectorizer__binary': [True, False],
        'vectorizer__max_df': [0.5, 1.0],
        'vectorizer__min_df': [1, 2],
    }
]

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=10, scoring='accuracy')
grid_search.fit(processed_training_data, training_labels)

# Get the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)


KeyboardInterrupt: 

In [29]:
pd.DataFrame({'body':lemmatized_training_data_swapped, 'subreddit': training_labels}).to_csv('lemmatized_training_data_swapped.csv', index=False)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

spacy_stopwords_list = list(fr_stop) + list(en_stop)
nltk_stopwords_list = stopwords.words('english') + stopwords.words('french')
other_stop_words = ['http', 'https', 'www']
stop_words = list(set().union(spacy_stopwords_list, nltk_stopwords_list, other_stop_words))

processed_training_data_1 = preprocess(lemmatized_training_data_swapped, stop_words = stop_words)
processed_training_data_2 = preprocess(training_data_swapped, stop_words = stop_words)

pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(max_df=0.5, min_df=2, token_pattern='(?u)\\b[a-z][a-z]+\\b')),
    ('model', LogisticRegression(random_state = seed, max_iter = 1000))
])

param_grid = [

    {
        'model__C': [1e-3, 1e-1, 1, 10, 1e2],
        'model__solver' : ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
    }
]

for x_train in (processed_training_data_1, processed_training_data_2):
  grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=10, scoring='accuracy')
  grid_search.fit(x_train, training_labels)

  # Get the best parameters and best score
  print("Best Parameters:", grid_search.best_params_)
  print("Best Score:", grid_search.best_score_)


In [29]:
spacy_stopwords_list = list(fr_stop) + list(en_stop)
nltk_stopwords_list = stopwords.words('english') + stopwords.words('french')
other_stop_words = ['http', 'https', 'www']

stop_words = list(set().union(spacy_stopwords_list, nltk_stopwords_list, other_stop_words))
processed_training_data = preprocess(lemmatized_training_data, stop_words = stop_words)
(training_data_split, testing_data_split, training_labels_split, testing_labels_split) = train_test_split(processed_training_data, training_labels, test_size = int(len(processed_training_data)*0.1), random_state=seed)
vct = TfidfVectorizer(token_pattern = r"(?u)\b[a-z][a-z]+\b", max_df = 0.5, max_features = 4000, min_df= 1)
training_data_split = vct.fit_transform(training_data_split)
testing_data_split = vct.transform(testing_data_split)

temp = LogisticRegression(random_state = seed, max_iter = 1000, C = 10)
temp.fit(training_data_split, training_labels_split)
y_pred =temp.predict(testing_data_split)
accuracy_score(y_pred, testing_labels_split)

0.6834532374100719

In [30]:
test_df   = pd.read_csv(path + 'test.csv', encoding = "ISO-8859-1")
test_data = test_df['body'].apply(lambda x: x.lower().replace('_', ' ').replace('\x82', 'e').replace('\x85', 'a').replace('\x87', 'c').replace('\x8a', 'e').replace('\x88', 'e').replace('\x93', 'a').replace('\x80', 'l').replace('\x96', 'o')).to_numpy()

spacy_stopwords_list = list(fr_stop) + list(en_stop)
nltk_stopwords_list = stopwords.words('english') + stopwords.words('french')
other_stop_words = ['http', 'https', 'www']
stop_words = list(set().union(spacy_stopwords_list, nltk_stopwords_list, other_stop_words))

processed_training_data = preprocess(lemmatized_training_data, stop_words = stop_words)
processed_test_data     = preprocess(test_data    , stop_words = None, is_lemmatize = True)
processed_test_data     = preprocess(processed_test_data, stop_words = stop_words)

vct = TfidfVectorizer(token_pattern = r"(?u)\b[a-z][a-z]+\b", max_df = 0.5, max_features = 4000, min_df= 1)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(processed_test_data)

pd.DataFrame({'Subreddit':y_pred}).reset_index().rename(columns={"index": "Id"}).to_csv('results.csv', index=False)

# Experimentation

In [ ]:
spacy_stopwords_list = list(fr_stop) + list(en_stop)
is_lemmatize = True ()
n_grams = (1,2)
alpha = 1 (default)
max_feat = 5000

In [9]:
# Set Hps
spacy_stopwords_list = list(fr_stop) + list(en_stop)
nltk_stopwords_list = stopwords.words('english') + stopwords.words('french')
other_stop_words = ['http', 'https', 'www']
stop_words = list(set().union(spacy_stopwords_list, nltk_stopwords_list, other_stop_words))

stop_words_list = [spacy_stopwords_list, nltk_stopwords_list, stop_words]
max_feats_list  = [3000, 4000, 5000]
is_lemmatize    = [True, False]
ng_list         = [(1,1), (1,2), (1,3)]
alphas          = [1, 2, 3]

hps = list(product(stop_words_list, max_feats_list, is_lemmatize, ng_list, alphas))

error = []
infos  = []

count = 1
# Evaluate
for stp_wd, max_feat, is_lem, ngram, alpha in hps:
  print('Iteration: {}'.format(count))
  if is_lem:
    processed_training_data = preprocess(lemmatized_training_data_swapped, stop_words = stp_wd)
  else:
    processed_training_data = preprocess(training_data_swapped, stop_words = stp_wd)

  model = NB(max_features = max_feat, ngram_range = ngram, alpha = alpha)

  mean_error, info = k_fold_validation(processed_training_data, training_labels, model = model)
  error.append(mean_error)
  infos.append(info)
  count += 1
  # print()
  # print('Mean Error: {}'.format(mean_error))
  # print()
  # print('\nModel Misses: {}'.format(model.no_pred))



Iteration: 1


Loading…: 100%|██████████████████████████| 139/139 [00:03<00:00, 41.63it/s]


Iteration: 2


Loading…: 100%|██████████████████████████| 139/139 [00:03<00:00, 43.44it/s]


Iteration: 3


...

Loading…: 100%|██████████████████████████| 139/139 [00:07<00:00, 17.48it/s]


Iteration: 161


Loading…: 100%|██████████████████████████| 139/139 [00:10<00:00, 13.11it/s]


Iteration: 162


Loading…: 100%|██████████████████████████| 139/139 [00:09<00:00, 14.10it/s]


In [10]:
error

[0.3697841726618705,
 0.3762589928057555,
 0.3899280575539569,
 0.3762589928057555,
 0.3776978417266187,
 0.3920863309352518,
 0.38057553956834533,
 0.39280575539568346,
 0.4057553956834532,
 0.36618705035971216,
 0.37194244604316545,
 0.3733812949640288,
 0.37266187050359717,
 0.37410071942446044,
 0.3884892086330935,
 0.38201438848920866,
 0.3942446043165468,
 0.4014388489208633,
 0.36978417266187047,
 0.3769784172661871,
 0.3892086330935252,
 0.36546762589928067,
 0.3705035971223022,
 0.3906474820143885,
 0.37338129496402883,
 0.38705035971223023,
 0.4043165467625899,
 0.36187050359712225,
 0.36906474820143886,
 0.38057553956834533,
 0.36834532374100715,
 0.37338129496402883,
 0.3906474820143885,
 0.36834532374100715,
 0.3798561151079137,
 0.3956834532374101,
 0.36978417266187047,
 0.37769784172661874,
 0.38920863309352516,
 0.356115107913669,
 0.3748201438848921,
 0.39424460431654673,
 0.3654676258992806,
 0.38633093525179857,
 0.4057553956834533,
 0.35683453237410073,
 0.364028776

In [11]:
import pickle

with open('error.pkl', 'wb') as file:
  pickle.dump(error, file)

with open('infos.pkl', 'wb') as file:
  pickle.dump(infos, file)

In [12]:
min(error)

0.356115107913669

In [13]:
np.argmin(error)

39

In [14]:
error[np.argmin(error)]

0.356115107913669

In [20]:
hps[np.argmin(error)]

(['étaient',
  'troisième',
  'ton',
  'cette',
  'dont',
  'certains',
  'toutes',
  'debout',
  'malgre',
  'onzième',
  'laisser',
  'celui',
  'troisièmement',
  'n’',
  'auront',
  'lequel',
  'seules',
  'suis',
  'parle',
  "c'",
  'lui-même',
  'â',
  'ouvert',
  'quelque',
  'dix-neuf',
  'neanmoins',
  'possible',
  'voila',
  'hormis',
  'également',
  'apres',
  'elle-même',
  'néanmoins',
  'devant',
  'dans',
  'suivante',
  'vu',
  'retour',
  'tente',
  'faisaient',
  'ouverte',
  'vers',
  'merci',
  'celles-ci',
  'parce',
  'm’',
  'certain',
  'être',
  'specifiques',
  'permet',
  'assez',
  'gens',
  'semble',
  'longtemps',
  'c’',
  'exactement',
  'de',
  'parler',
  'memes',
  'telles',
  'unes',
  'mais',
  'soi',
  'semblaient',
  'cinq',
  'selon',
  'même',
  'déja',
  'car',
  'bat',
  'celle-là',
  'celles-la',
  'revoici',
  'va',
  'nouveau',
  'tel',
  'ne',
  'seront',
  'déjà',
  'vos',
  'suivantes',
  'devers',
  'revoila',
  'directe',
  'mainten

In [19]:
len(hps[np.argmin(error)][0])

833

In [ ]:
len()

In [25]:
# With Processing

hps = list(product(stop_words_list, max_feats_list, is_lemmatize, ng_list, alphas))

stop_words, max_feat, is_lem, ng, alpha = hps[np.argmin(error)]

if is_lem:
  processed_training_data = preprocess(lemmatized_training_data_swapped, stop_words = stop_words)
else:
  processed_training_data = preprocess(training_data_swapped, stop_words = stop_words)

test_split = 0.2

(training_data_split, testing_data_split, training_labels_split, testing_labels_split) = train_test_split(processed_training_data, training_labels, test_size = test_split, random_state=seed)

model = NB(max_features = max_feat, ngram_range = ng, alpha = alpha)
model.fit(training_data_split, training_labels_split)
predictions = model.predict(testing_data_split)
accuracy_score(testing_labels_split, predictions)

Loading…: 100%|██████████████████████████| 280/280 [00:20<00:00, 13.69it/s]


0.5892857142857143

In [13]:
sorted(model.word_count.items(), key = lambda x: x[1], reverse=True)

[('time', 177),
 ('year', 169),
 ('place', 135),
 ('london', 113),
 ('way', 104),
 ('find', 97),
 ('live', 96),
 ('come', 94),
 ('try', 94),
 ('day', 91),
 ('city', 86),
 ('feel', 79),
 ('new', 78),
 ('best', 72),
 ('bad', 71),
 ('help', 71),
 ('old', 71),
 ('great', 70),
 ('bit', 67),
 ('pay', 67),
 ('friend', 65),
 ('faire', 64),
 ('area', 63),
 ('home', 62),
 ('life', 62),
 ('walk', 62),
 ('big', 61),
 ('pari', 61),
 ('right', 61),
 ('start', 61),
 ('house', 59),
 ('long', 59),
 ('paris', 59),
 ('sure', 59),
 ('job', 58),
 ('point', 58),
 ('experience', 57),
 ('train', 57),
 ('ago', 56),
 ('question', 56),
 ('love', 55),
 ('bien', 54),
 ('end', 54),
 ('let', 53),
 ('money', 53),
 ('probably', 53),
 ('street', 52),
 ('ask', 50),
 ('guy', 50),
 ('montreal', 50),
 ('actually', 49),
 ('line', 49),
 ('maybe', 49),
 ('month', 49),
 ('change', 48),
 ('little', 48),
 ('living', 48),
 ('metro', 48),
 ('week', 48),
 ('family', 47),
 ('non', 47),
 ('small', 47),
 ('price', 46),
 ('close', 45),

In [ ]:
# With Processing

spacy_stopwords_list = list(fr_stop) + list(en_stop)
nltk_stopwords_list = stopwords.words('english') + stopwords.words('french')
other_stop_words = ['http', 'https', 'www']
stop_words = list(set().union(spacy_stopwords_list, nltk_stopwords_list, other_stop_words))

processed_training_data = preprocess(training_data, stop_words = stop_words)

test_split = 0.1

(training_data_split, testing_data_split, training_labels_split, testing_labels_split) = train_test_split(processed_training_data, training_labels, test_size = int(len(processed_training_data)*test_split), random_state=seed)

model = NB()
model.fit(training_data_split, training_labels_split)
mean_error, info = k_fold_validation(training_data_split, training_labels_split, model = model)
predictions = model.predict(testing_data_split)
print(1-mean_error)
accuracy_score(testing_labels_split, predictions)

(1, 1)


Loading…: 100%|██████████████████████████| 126/126 [00:02<00:00, 42.02it/s]


(1, 1)


Loading…: 100%|██████████████████████████| 126/126 [00:02<00:00, 44.37it/s]


(1, 1)


...

Loading…: 100%|██████████████████████████| 126/126 [00:02<00:00, 45.99it/s]


(1, 1)


Loading…: 100%|██████████████████████████| 139/139 [00:03<00:00, 43.98it/s]

0.6158730158730159


0.6690647482014388

# Actual Test Case

In [ ]:
test_df   = pd.read_csv(path + 'test.csv', encoding = "ISO-8859-1")
test_data = test_df['body'].apply(lambda x: x.lower().replace('_', ' ')).to_numpy()

stop_words, max_feat, _, _ = hps[220]

processed_training_data = preprocess(lemmatized_training_data, stop_words = stop_words)
processed_test_data     = preprocess(test_data    , stop_words = stop_words, is_lemmatize = True)

model = NB(max_features = max_feat, ngram_range = (1,1))
model.fit(processed_training_data, training_labels)
predictions = model.predict(processed_test_data)

pd.DataFrame({'Subreddit':predictions}).reset_index().rename(columns={"index": "Id"}).to_csv('results.csv', index=False)

Loading…: 100%|██████████████████████████| 600/600 [00:17<00:00, 33.66it/s]


In [ ]:
pd.DataFrame({'Subreddit':predictions}).reset_index().rename(columns={"index": "Id"}).to_csv('results.csv', index=False)